<a href="https://colab.research.google.com/github/7ft10/JiraExporter/blob/main/DataFlows - Server/Jira_Lookups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#! Library Import

import os 
import importlib.util

if importlib.util.find_spec("pandas") is None:	
	os.system("pip install pandas")

if importlib.util.find_spec("dotenv") is None:	
	os.system("pip install --quiet openai python-dotenv")

In [2]:
#! pandas Config

import pandas as pd
import requests
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

pd.options.mode.use_inf_as_na = True
#pd.set_option("display.max_rows", 10)
#pd.set_option("display.expand_frame_repr", True)
#pd.set_option('display.width', 1000)
#pd.options.display.max_seq_items = 200000
#pd.options.display.max_rows = 10

In [3]:
#! Parameters

import os 
import dotenv
import importlib.util

if importlib.util.find_spec("google.colab") is not None: ## if using google colab
    if not os.path.exists('.env'):
        from google.colab import files
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        try:
            os.rename(file_name, '.env')
        except:
            pass

dotenv.load_dotenv('.env')

Host = os.getenv('SECRETS_HOST')
Username = os.getenv('SECRETS_USERNAME')
Password = os.getenv('SECRETS_PASSWORD')
ValidProjectCategories = ["'Portfolio'"]

if Host is None:
    raise Exception("Secrets not found")

display("Host: " + Host)
display("ValidProjectCategories: " + ','.join(ValidProjectCategories))

'Host: https://jira.budgetdirect.com.au/'

"ValidProjectCategories: 'Portfolio'"

In [4]:
#! Functions

import base64
import pandas as pd
import re 
import requests
import warnings
from functools import reduce
    
def _ExpandColumn(self:pd.DataFrame, colName:str, columnsToExpand = [], prefix:str = "Prefix", sentenceCase:bool = True) -> pd.DataFrame:
    if (prefix == "Prefix"):
        prefix = colName + " "
        with warnings.catch_warnings():
          warnings.simplefilter(action='ignore', category=FutureWarning)
          expandedCols = self[colName].apply(lambda x: pd.Series(x).add_prefix(prefix))
        columnsToExpand = [prefix + c for c in columnsToExpand]
    else:
        expandedCols = self[colName].apply(lambda x: pd.Series(x))
    
    if len(columnsToExpand) > 0:        
        expandedCols = expandedCols[columnsToExpand]
    
    if sentenceCase:
        expandedCols.columns = [fnSentenceCase(c) for c in expandedCols.columns] 

    return pd.concat([self.drop(colName, axis=1), expandedCols], axis=1)

pd.DataFrame.expand = _ExpandColumn

def fnSentenceCase(s):
    s = (' '.join(dict.fromkeys(s.split())))  # remove duplicate words
    s = s.replace("0", "") # remove "0" 
    s = s.strip()
    return ' '.join([x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]) # sentence case

def _SentenceCaseColumns(self:pd.DataFrame) -> pd.DataFrame: 
    self.columns = [fnSentenceCase(c) for c in self.columns] 
    return self

pd.DataFrame.sentence_case_columns = _SentenceCaseColumns

def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic " + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120"
    }

def fnSearch(jql, fields = None, expand = None):
    def ApiCall(startAt) :
        url = "/rest/api/latest/search"
        headers = fnGetDefaultHeaders()
        defaultContents = {
            "startAt": startAt,
            "maxResults": "2",
            "jql": jql
        }
        if fields is not None:
            defaultContents["fields"] = fields.tolist()
        if expand is not None and expand != "":
            defaultContents["expand"] = expand        
        response = requests.post(Host + url, headers = headers, json = defaultContents, verify=False)
        return response.json()
    values = fnAPI(ApiCall)
    if len(values.index) > 1:
        return values
    else:
        None

def fnGetIssueTypeFields(IssueTypes) -> pd.DataFrame:
    def ApiCall(startAt) :
        url = "rest/api/latest/issue/createmeta"
        headers = fnGetDefaultHeaders()
        params = {
            "expand": "projects.issuetypes.fields",
            "projectKeys": ','.join(fnGetValidProjectKeys()["key"].values),
            "issuetypeNames": ','.join(IssueTypes).replace("'", "")
        }
        response = requests.get(Host + url, headers = headers, params = params, verify=False)
        return response.json()
    
    df = fnAPI(ApiCall)
    df = df.drop(["expand"], axis=1)
    df = df.explode("projects")
    df = df.expand("projects", [], None, False)    
    try:
        df = df[["issuetypes"]]
    except: 
        raise Exception("No issue metadata - check the valid project categories are correct")
    df = df.explode("issuetypes")
    df = df.expand("issuetypes", [], None, False)
    df = df[["fields"]]
    df = df.expand("fields", [], None, False)
    df = df.loc[:,~df.columns.duplicated()]

    values = []
    for x in df.columns:        
        try:            
            valid:pd.DataFrame = pd.DataFrame( df[~df[x].isnull()] )[[x]].iloc[0].get(0)
            values.append({
                "fieldId": valid['key'] if "key" in valid else valid["fieldId"],
                "name": valid['name'],
                "schema_type": valid['schema']['type'],
                "required": valid['required']
            })
        except:
            #display(x)
            pass
    values.append({ "fieldId": 'status', "name": 'Status', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'created', "name": 'Created', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'updated', "name": 'Updated', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'resolution', "name": 'Resolution', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'resolutiondate', "name": 'Resolution Date', "schema_type": 'date', "required": False })
    values.append({ "fieldId": 'lastViewed', "name": 'Last Viewed', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'id', "name": 'Id', "schema_type": 'number', "required": True })
    values.append({ "fieldId": 'key', "name": 'Key', "schema_type": 'string', "required": True })
    df = pd.DataFrame(values)
    df = df.drop_duplicates().sort_values("fieldId")
    return df 

def fnGetValidProjectKeys() -> pd.DataFrame:
    def ApiCall(startAt) :
        url = "/rest/api/latest/project"
        headers = fnGetDefaultHeaders()
        params = { }
        response = requests.get(Host + url, headers = headers, params = params, verify=False)
        return response.json()
    df = fnAPI(ApiCall)
    df = df.expand("projectCategory")
    if 'ValidProjectCategories' in globals() and len(ValidProjectCategories) > 0:
        df = df.loc[df['Project Category Name'].isin(ValidProjectCategories) | ("'" + df['Project Category Name'] + "'").isin(ValidProjectCategories)]
    return df[["key"]]

def fnAPI(webRequestDelegate, startAt = 0) -> pd.DataFrame:
    def flatten_reduce_lambda(frm):
        try:
            return list(reduce(lambda x, y: x + y, frm, []))         
        except:
            return list(reduce(lambda x, y: x + y, [frm], [])) 
    def innerGetResults(webRequestDelegate, startAt = 0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(webRequestDelegate, startAt + results["maxResults"])
            else:
                return [results]
        else:
            return [results]
    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df


In [7]:
#! Jira Lookups Capture

from IPython.display import display

def ApiCall(startAt) :
	url = "rest/api/latest/issue/createmeta"
	headers = fnGetDefaultHeaders()
	params = {
		"expand": "projects.issuetypes.fields"
	}
	response = requests.get(Host + url, headers = headers, params = params, verify=False)
	return response.json()

df = fnAPI(ApiCall)
df = df.drop(["expand"], axis=1)
df = df.explode("projects")
df = df.expand("projects", [], None, False)
df = df[["issuetypes"]]
df = df.explode("issuetypes")
df = df.expand("issuetypes", [], None, False)
df = df[["fields"]]
df = df.expand("fields", [], None, False)

goldenDF:pd.DataFrame = df.copy(deep=True)
globals()['goldenDF'] = goldenDF 

display(goldenDF.dtypes)
display(goldenDF.head())

summary              object
reporter             object
customfield_17070    object
customfield_20071    object
description          object
                      ...  
customfield_21110    object
customfield_21112    object
customfield_21113    object
customfield_21114    object
customfield_21109    object
Length: 381, dtype: object

,summary,reporter,customfield_17070,customfield_20071,description,labels,customfield_19988,customfield_17080,customfield_21146,customfield_21148,...,customfield_15091,customfield_19895,customfield_21116,customfield_21118,customfield_15773,customfield_21110,customfield_21112,customfield_21113,customfield_21114,customfield_21109
0,"{'required': True, 'schema': {'type': 'string'...","{'required': True, 'schema': {'type': 'user', ...","{'required': True, 'schema': {'type': 'option'...","{'required': False, 'schema': {'type': 'array'...","{'required': False, 'schema': {'type': 'string...","{'required': False, 'schema': {'type': 'array'...","{'required': False, 'schema': {'type': 'user',...","{'required': False, 'schema': {'type': 'array'...","{'required': False, 'schema': {'type': 'array'...","{'required': False, 'schema': {'type': 'array'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"{'required': True, 'schema': {'type': 'string'...","{'required': True, 'schema': {'type': 'user', ...",NaN,NaN,"{'required': False, 'schema': {'type': 'string...","{'required': False, 'schema': {'type': 'array'...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"{'required': True, 'schema': {'type': 'string'...","{'required': True, 'schema': {'type': 'user', ...",NaN,NaN,"{'required': False, 'schema': {'type': 'string...","{'required': False, 'schema': {'type': 'array'...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"{'required': True, 'schema': {'type': 'string'...","{'required': True, 'schema': {'type': 'user', ...",NaN,NaN,"{'required': False, 'schema': {'type': 'string...","{'required': False, 'schema': {'type': 'array'...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"{'required': True, 'schema': {'type': 'string'...","{'required': True, 'schema': {'type': 'user', ...","{'required': True, 'schema': {'type': 'option'...","{'required': False, 'schema': {'type': 'array'...","{'required': False, 'schema': {'type': 'string...","{'required': False, 'schema': {'type': 'array'...","{'required': False, 'schema': {'type': 'user',...","{'required': False, 'schema': {'type': 'array'...","{'required': False, 'schema': {'type': 'array'...","{'required': False, 'schema': {'type': 'array'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#! Additional Functions for this type 

def fnGetAllowedValuesForField(field:str, columns):
    df = None
    if 'goldenDF' not in globals() or goldenDF is None: 
        display("Base data frame not loaded") 
    else:
        df:pd.DataFrame = goldenDF.copy(deep = True)
        df = df[[field]]
        if df is None or len(df.index) == 0:	
            display("No results")
        else:
            try:
                df = df.expand(field, ["hasDefaultValue", "allowedValues", "defaultValue"], None, False)
            except:
                df = df.expand(field, ["hasDefaultValue", "allowedValues"], None, False)

            df = df[~df["allowedValues"].isna()]
            df = df[df["allowedValues"].map(lambda d: len(d)) > 0]
            if df is None or len(df.index) == 0:	
                display("No Allowed Values")
            else:
                df = df.explode("allowedValues")
                df = df.expand("allowedValues", columns, None, False)

                if "defaultValue" in df.columns.values:
                    df = df.expand("defaultValue", columns)				

                if "id" in df.columns.values:
                    df = df[~df["id"].isna()]
                    
                df = df.drop_duplicates() 

                def check_if_default(row) -> bool:
                    columns = row.axes[0].values
                    if "hasDefaultValue" in columns:
                        if "id" in columns and "Default Value Id" in columns:
                            return row["hasDefaultValue"] and row["id"] == row["Default Value Id"]
                        return row["hasDefaultValue"]
                    return False
                
                df["Is Default"] = df.apply ( check_if_default, axis=1)
                
                df = df[ columns + ["Is Default"] ]
                
                if "name" in df.columns.values:
                    df = df.sort_values("name")
                if "value" in df.columns.values:
                    df = df.sort_values("value")
                    
                if "id" in df.columns.values:
                    df["id"] = df["id"].astype('Int64')
              
                df = df.drop_duplicates() 
                df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)								            

    return df


def fnGetLookupsForIssueType(issueTypesList) -> dict:
    df = fnGetIssueTypeFields(issueTypesList)

    lookups = {}

    for _, row in df.iterrows(): 
        field = row["fieldId"]
        name = row["name"]
        type = row["schema_type"]
        if type in ["option", "option2"]:
            df2 = None
            try:
                df2 = fnGetAllowedValuesForField(field, ["id", "value"])
            except:
                pass
            try:
                df2 = fnGetAllowedValuesForField(field, ["id", "name"])
            except:
                pass
            if df2 is not None:
                lookups[field] = [ name,  df2 ]
                                
    return lookups


In [10]:
#! Lookups Task

from IPython.display import display

lookups = fnGetLookupsForIssueType(["Task"])

if len(lookups) == 0:
    display("No lookups")
else:
	for field, lookup in lookups.items():
		name, df2 = lookup
		display(field + " -> " + name)
		display(df2.dtypes)
		display(df2.head())

'customfield_15082 -> Risk Probability'

Id                     Int64
Value         string[python]
Is Default           boolean
dtype: object

,Id,Value,Is Default
0,17348,Almost Certain,False
1,17349,Likely,False
2,17350,Possible,False
3,17352,Rare,True
4,17351,Unlikely,False


'customfield_15083 -> Change is'

Id                     Int64
Value         string[python]
Is Default           boolean
dtype: object

,Id,Value,Is Default
0,17354,Mandatory,False
1,17353,Optional,True


'customfield_16972 -> T-Shirt Size'

Id                     Int64
Value         string[python]
Is Default           boolean
dtype: object

,Id,Value,Is Default
0,25891,Difficult ($474K - $947K),False
1,20475,L,False
2,20474,M,False
3,25890,Moderate ($236K - $474K),False
4,20473,S,False
5,25889,Simple (0 - 236K),False
6,25892,Very Complex (947K - $1.4M),False
7,20477,We dont know enough,False
8,20476,XL,False
9,20472,XS,False


In [11]:
#! Lookups Epics

from IPython.display import display

lookups = fnGetLookupsForIssueType(["Epic"])

if len(lookups) == 0:
    display("No lookups")
else:
	for field, lookup in lookups.items():
		name, df2 = lookup
		display(field + " -> " + name)
		display(df2.dtypes)
		display(df2.head())

'customfield_15082 -> Risk Probability'

Id                     Int64
Value         string[python]
Is Default           boolean
dtype: object

,Id,Value,Is Default
0,17348,Almost Certain,False
1,17349,Likely,False
2,17350,Possible,False
3,17352,Rare,True
4,17351,Unlikely,False


'customfield_15083 -> Change is'

Id                     Int64
Value         string[python]
Is Default           boolean
dtype: object

,Id,Value,Is Default
0,17354,Mandatory,False
1,17353,Optional,True


'customfield_16972 -> T-Shirt Size'

Id                     Int64
Value         string[python]
Is Default           boolean
dtype: object

,Id,Value,Is Default
0,25891,Difficult ($474K - $947K),False
1,20475,L,False
2,20474,M,False
3,25890,Moderate ($236K - $474K),False
4,20473,S,False


In [12]:
#! Lookups Story

from IPython.display import display

lookups = fnGetLookupsForIssueType(["Story"])

if len(lookups) == 0:
    display("No lookups")
else:
	for field, lookup in lookups.items():
		name, df2 = lookup
		display(field + " -> " + name)
		display(df2.dtypes)
		display(df2.head())

'customfield_15082 -> Risk Probability'

Id                     Int64
Value         string[python]
Is Default           boolean
dtype: object

,Id,Value,Is Default
0,17348,Almost Certain,False
1,17349,Likely,False
2,17350,Possible,False
3,17352,Rare,True
4,17351,Unlikely,False


'customfield_15083 -> Change is'

Id                     Int64
Value         string[python]
Is Default           boolean
dtype: object

,Id,Value,Is Default
0,17354,Mandatory,False
1,17353,Optional,True


'customfield_16972 -> T-Shirt Size'

Id                     Int64
Value         string[python]
Is Default           boolean
dtype: object

,Id,Value,Is Default
0,25891,Difficult ($474K - $947K),False
1,20475,L,False
2,20474,M,False
3,25890,Moderate ($236K - $474K),False
4,20473,S,False
